# A movie recommendation system

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix

Download the movielens dataset [here](http://files.grouplens.org/datasets/movielens/ml-20m.zip) 

** Before loading in the data, I highly recommend cutting out some of the "ratings.csv" file. It's 20M rows long and can take a long time to process. In bash you can do something like: **

`cat ratings.csv | head -100000 > ratings_small.csv` 

We'll also load in the movies.csv file to a DataFrame - this will act as a dictionary to translate between MovieID and Movie Titles.

In [4]:
movies = pd.read_table('ml-20m/movies.csv', sep=',',names = ['movieId',"Title","genres"], skiprows=1)

In [5]:
movies

,movieId,Title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


We'll use numpy's text reader to create a series of lists that we'll use to create a sparse matrix. In this case, we'll have a coordinate based sparse matrix creation.

In [7]:
u, m, r, junk = np.loadtxt('ml-20m/ratings_small.csv', delimiter=',', skiprows=1).T
mat = coo_matrix((r, (u-1, m-1)), shape=(u.max(), m.max()))

In [8]:
print(mat.toarray()[0][28])
print(mat.shape)

3.5
(702, 128594)


### Aside: What's a Sparse Matrix and why do we like it?

So when you think about this dataset, we know there are ~130,000 movies. That means that most of our users won't have seen most of the movies. Even if we had one user that really binged watched movies all the time, they might have seen 5,000 movies - so there are 125,000 movies where we have no entry for that user. If we wanted to store that matrix directly we'd have a matrix that looks something like:

In [9]:
a = np.zeros((100,100))
a[5,5] = 1
pd.DataFrame(a)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We've essentially stored a bunch of zeros. That's a huge waste of memory. Instead, we could store something like this:

(5, 5, 1)

Meaning in the 5th row and 5th column, make the value 1. Make everything else a zero. That saves us a TON of space.

Now we can use Scipy's excellent SVDS method to decompose the sparse matrix into 3 matrices via Singular Value Decomposition (the extra 'S' stands for sparse). 

In [10]:
from scipy.sparse.linalg import svds
U, s, VT = svds(mat,k=100)

Hooray! Now we have the things. So now we can find similar movies. 

In [11]:
def find_similar_movies(itemID, VT, num_recom=2):
    recs = []
    for item in range(VT.T.shape[0]):
        if item != itemID:
            recs.append([item+1,np.dot(VT.T[itemID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

In [12]:
movies[movies.Title == "Toy Story 2 (1999)"]

,movieId,Title,genres
3027,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy


In [18]:
MOVIE_ID = 3114
for item in find_similar_movies(MOVIE_ID,VT,num_recom=10):
    print(item)
    print(movies[movies['movieId'] == int(item[0])].Title,'\n')

(3114, 0.1245277382352756)
3027    Toy Story 2 (1999)
Name: Title, dtype: object 

(1, 0.096984857294616561)
0    Toy Story (1995)
Name: Title, dtype: object 

(2355, 0.043104443630875247)
2270    Bug's Life, A (1998)
Name: Title, dtype: object 

(2987, 0.041949127538016634)
2901    Who Framed Roger Rabbit? (1988)
Name: Title, dtype: object 

(6377, 0.040522854363774487)
6271    Finding Nemo (2003)
Name: Title, dtype: object 

(4886, 0.035061740395502361)
4790    Monsters, Inc. (2001)
Name: Title, dtype: object 

(1148, 0.03103362609236119)
1125    Wallace & Gromit: The Wrong Trousers (1993)
Name: Title, dtype: object 

(2291, 0.030527541370676318)
2206    Edward Scissorhands (1990)
Name: Title, dtype: object 

(78499, 0.026076575060862393)
15401    Toy Story 3 (2010)
Name: Title, dtype: object 

(551, 0.025716728078137858)
547    Nightmare Before Christmas, The (1993)
Name: Title, dtype: object 



We can also look for movies that have the most overlap in the 100 dimensional space with a given user, since they're both in that same 100 dimensional space. So here we calculate the overlap between all items and the input user.

In [19]:
def get_recommends_for_user(userID, U, VT, num_recom=2):
    recs = []
    for item in range(VT.T.shape[0]):
        recs.append([item+1,np.dot(U[userID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

USERID=25
for item in get_recommends_for_user(USERID,U,VT,10):
    print(item)
    print("Actual Rating: ", mat.toarray()[USERID][item[0]])
    print(movies[movies.movieId == int(item[0])].Title,'\n')

(356, 0.045122506917985097)
Actual Rating:  5.0
352    Forrest Gump (1994)
Name: Title, dtype: object 

(1961, 0.040241276842522003)
Actual Rating:  0.0
1877    Rain Man (1988)
Name: Title, dtype: object 

(1270, 0.038155112900624247)
Actual Rating:  0.0
1242    Back to the Future (1985)
Name: Title, dtype: object 

(1097, 0.03551559800834489)
Actual Rating:  0.0
1075    E.T. the Extra-Terrestrial (1982)
Name: Title, dtype: object 

(1307, 0.032959828597354551)
Actual Rating:  0.0
1278    When Harry Met Sally... (1989)
Name: Title, dtype: object 

(47, 0.032932686491077973)
Actual Rating:  0.0
46    Seven (a.k.a. Se7en) (1995)
Name: Title, dtype: object 

(597, 0.031553835377081695)
Actual Rating:  0.0
591    Pretty Woman (1990)
Name: Title, dtype: object 

(1198, 0.029896968667501522)
Actual Rating:  0.0
1173    Raiders of the Lost Ark (Indiana Jones and the...
Name: Title, dtype: object 

(1968, 0.029516697365152389)
Actual Rating:  0.0
1884    Breakfast Club, The (1985)
Name: Title,

What if we find a user that's the most closely related to the current user, then recommend all the movies the "closest user" likes that our current user hasn't seen? 

**Note: this uses coo_matrix.toarray() and therefore creates a HUGE dense matrix for lookup purposes, you probably wouldn't want to do that normally; but it works for only a few 100k rows.**

In [20]:
def get_recommends_user(userID, U, df):
    userrecs = []
    for user in range(U.shape[0]):
        if user!= userID:
            userrecs.append([user,np.dot(U[userID],U[user])])
    final_rec = [i[0] for i in sorted(userrecs,key=lambda x: x[1],reverse=True)]
    comp_user = final_rec[0]
    print("User #%s's most similar user is User #%s "% (userID, comp_user))
    rec_likes = df.iloc[comp_user]
    current = df.iloc[userID]
    recs = []
    for i,item in enumerate(current):
        if item != rec_likes[i] and rec_likes[i]!=0:
            recs.append(i)
    return recs

user_to_rec = 3
print("Items for User %s to check out: "% user_to_rec, get_recommends_user(user_to_rec,U,pd.DataFrame(mat.toarray())))

User #3's most similar user is User #282 
Items for User 3 to check out:  [5, 9, 15, 24, 28, 31, 33, 35, 46, 49, 94, 109, 110, 140, 149, 164, 171, 184, 193, 197, 207, 230, 231, 234, 246, 259, 287, 289, 295, 299, 306, 307, 315, 317, 328, 336, 343, 348, 355, 356, 379, 433, 434, 453, 456, 467, 470, 479, 480, 491, 499, 506, 519, 526, 538, 540, 550, 579, 588, 589, 591, 592, 596, 607, 647, 664, 713, 719, 732, 735, 777, 779, 783, 785, 834, 857, 903, 911, 912, 922, 923, 952, 967, 1035, 1041, 1046, 1078, 1079, 1088, 1089, 1093, 1096, 1119, 1126, 1128, 1134, 1135, 1172, 1174, 1175, 1185, 1188, 1192, 1195, 1197, 1199, 1200, 1205, 1207, 1209, 1213, 1214, 1215, 1218, 1220, 1221, 1222, 1224, 1229, 1233, 1239, 1240, 1244, 1245, 1246, 1255, 1257, 1258, 1259, 1260, 1262, 1269, 1274, 1275, 1280, 1287, 1290, 1297, 1319, 1345, 1347, 1349, 1353, 1369, 1386, 1392, 1393, 1465, 1484, 1499, 1512, 1526, 1543, 1572, 1579, 1586, 1616, 1640, 1672, 1679, 1689, 1692, 1700, 1703, 1720, 1728, 1731, 1747, 1778, 1808, 1

# svdRec - A class to do the job

In [21]:
import numpy as np
from scipy.sparse import coo_matrix,csr_matrix
from scipy.sparse.linalg import svds

class svdRec():
    def __init__(self):
        self.U, self.s, self.V = (None,None,None)
        self.user_encoder = None
        self.item_encoder = None
        self.mat = None
        self.decomp = False
    
    def load_csv_sparse(self,filename,delimiter=',',skiprows=None, is_one_indexed=True):
        print("Note: load_csv_sparse expects a csv in the format of: rowID, colID, Value, ...")
        u, m, r = np.loadtxt(filename, delimiter=delimiter, skiprows=skiprows, usecols=(0,1,2)).T
        if is_one_indexed:
            self.mat = coo_matrix((r, (u-1, m-1)), shape=(u.max(), m.max())).tocsr()
        else:
            self.mat = coo_matrix((r, (u, m)), shape=(u.max(), m.max())).tocsr()
        print("Created matrix of shape: ",self.mat.shape)
        
    def load_data_numpy(self, array, data_type=float):
        self.mat = csr_matrix(array,dtype=data_type)
        print("Created matrix of shape: ",self.mat.shape)
        
    def load_item_encoder(self, d):
        if type(d) != dict:
            raise TypeError("Encoder must be dictionary with key = itemID and value = Title")
        self.item_encoder = d
        
    def load_user_encoder(self, d):
        if type(d) != dict:
            raise TypeError("Encoder must be dictionary with key = userID and value = Title")
        self.user_encoder = d
        
    def get_item_name(self,itemid):
        if self.item_encoder:
            return self.item_encoder[str(itemid)]
        else:
            return "No ItemId -> Item-name Encoder Built!"
    
    def get_user_name(self,userid):
        if self.item_encoder:
            return self.user_encoder[str(userid)]
        else:
            return "No UserID -> Username Encoder Built!"
    
    def SVD(self, num_dim=None):
        if num_dim==None:
            print("Number of SVD dimensions not requested, using %s dimensions." % (min(self.mat.shape)-1), "To set, use num_dim.")
            num_dim = min(self.mat.shape)-1
        self.U, self.s, self.VT = svds(self.mat,k=num_dim)
        self.decomp = True
    
    def get_cell(self,i,j):
        return self.mat[1,:].toarray()[0,j]
    
    def get_similar_items(self, itemID, num_recom=5, show_similarity=False):
        if not self.decomp:
            raise ValueError("Must run SVD() before making recommendations!")
        recs = []
        for item in range(self.VT.T.shape[0]):
                recs.append([item+1,self.item_similarity(itemID-1,item)])
        if show_similarity:
            final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
        else:
            final_rec = [i[0] for i in sorted(recs,key=lambda x: x[1],reverse=True)]
        return final_rec[:num_recom]
    
    def item_similarity(self,item1,item2):
        if not self.decomp:
            raise ValueError("Must run SVD() before making recommendations!")
        return np.dot(self.VT.T[item1],self.VT.T[item2])
    
    def user_similarity(self,user1,user2):
        if not self.decomp:
            raise ValueError("Must run SVD() before making recommendations!")
        return np.dot(self.U[user1],self.U[user2])
    
    def user_item_similarity(self,user,item):
        if not self.decomp:
            raise ValueError("Must run SVD() before making recommendations!")
        return np.dot(self.U[user],self.VT.T[item])
    
    def user_item_predict(self,user,item):
        if not self.decomp:
            raise ValueError("Must run SVD() before making recommendations!")
        return np.dot(self.U[user],self.VT.T[item])
        
    def recommends_for_user(self, userID, num_recom=2, show_similarity=False):
        if not self.decomp:
            raise ValueError("Must run SVD() before making recommendations!")
        recs = []
        for item in range(self.VT.T.shape[0]):
            recs.append((item+1,self.user_item_predict(userID-1,item)))
        if show_similarity:
            final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
        else:
            final_rec = [i[0] for i in sorted(recs,key=lambda x: x[1],reverse=True)]
        return final_rec[:num_recom]
    
    def recs_from_closest_user(self, userID):
        if not self.decomp:
            raise ValueError("Must run SVD() before making recommendations!")
        userrecs = []
        for user in range(self.U.shape[0]):
            if user!= userID:
                userrecs.append([user,self.user_similarity(userID,user)])
        final_rec = [i[0] for i in sorted(userrecs,key=lambda x: x[1],reverse=True)]
        comp_user = final_rec[0]
        print("User #%s's most similar user is User #%s "% (userID, comp_user))
        data = self.mat.toarray()
        rec_likes = data[comp_user]
        current = data[userID]
        recs = []
        for i,item in enumerate(current):
            if item != rec_likes[i] and rec_likes[i]!=0:
                recs.append(i)
        return recs
        

In [22]:
svd = svdRec()
svd.load_csv_sparse('ml-20m/ratings_small.csv', delimiter=',', skiprows=1)
svd.SVD(num_dim=100)

Note: load_csv_sparse expects a csv in the format of: rowID, colID, Value, ...
Created matrix of shape:  (702, 128594)


In [24]:
movies = pd.read_table('ml-20m/movies.csv', sep=',',names = ['movieId',"Title","genres"])
movie_dict = {}
for i, row in movies.iterrows():
    movie_dict.update({row['movieId']: row['Title']})
svd.load_item_encoder(movie_dict)
print(svd.item_encoder['3114'])
print(svd.get_cell(1,2))

Toy Story 2 (1999)
4.0


In [25]:
MOVIE_ID = 3114
for item in svd.get_similar_items(MOVIE_ID,show_similarity=True):
    print(item)
    print(svd.get_item_name(item[0]),'\n')

(3114, 0.12452773823527577)
Toy Story 2 (1999) 

(1, 0.096984857294616172)
Toy Story (1995) 

(2355, 0.043104443630874983)
Bug's Life, A (1998) 

(2987, 0.041949127538016877)
Who Framed Roger Rabbit? (1988) 

(6377, 0.040522854363774341)
Finding Nemo (2003) 



In [26]:
USERID=25
for item in svd.recommends_for_user(USERID, num_recom=5, show_similarity=False):
    print("ID: ", item)
    print("Actual Rating: ", svd.mat.toarray()[USERID][item])
    print("Title: ",svd.get_item_name(item),'\n')

ID:  356
Actual Rating:  5.0
Title:  Forrest Gump (1994) 

ID:  1961
Actual Rating:  0.0
Title:  Rain Man (1988) 

ID:  1270
Actual Rating:  0.0
Title:  Back to the Future (1985) 

ID:  1097
Actual Rating:  0.0
Title:  E.T. the Extra-Terrestrial (1982) 

ID:  1307
Actual Rating:  0.0
Title:  When Harry Met Sally... (1989) 



In [27]:
user_to_rec = 3
print("Items for User %s to check out: "% user_to_rec, svd.recs_from_closest_user(user_to_rec))

User #3's most similar user is User #282 
Items for User 3 to check out:  [5, 9, 15, 24, 28, 31, 33, 35, 46, 49, 94, 109, 110, 140, 149, 164, 171, 184, 193, 197, 207, 230, 231, 234, 246, 259, 287, 289, 295, 299, 306, 307, 315, 317, 328, 336, 343, 348, 355, 356, 379, 433, 434, 453, 456, 467, 470, 479, 480, 491, 499, 506, 519, 526, 538, 540, 550, 579, 588, 589, 591, 592, 596, 607, 647, 664, 713, 719, 732, 735, 777, 779, 783, 785, 834, 857, 903, 911, 912, 922, 923, 952, 967, 1035, 1041, 1046, 1078, 1079, 1088, 1089, 1093, 1096, 1119, 1126, 1128, 1134, 1135, 1172, 1174, 1175, 1185, 1188, 1192, 1195, 1197, 1199, 1200, 1205, 1207, 1209, 1213, 1214, 1215, 1218, 1220, 1221, 1222, 1224, 1229, 1233, 1239, 1240, 1244, 1245, 1246, 1255, 1257, 1258, 1259, 1260, 1262, 1269, 1274, 1275, 1280, 1287, 1290, 1297, 1319, 1345, 1347, 1349, 1353, 1369, 1386, 1392, 1393, 1465, 1484, 1499, 1512, 1526, 1543, 1572, 1579, 1586, 1616, 1640, 1672, 1679, 1689, 1692, 1700, 1703, 1720, 1728, 1731, 1747, 1778, 1808, 1

**For the below, the scale is not particularly meaningful, but we can quickly compare similarities (which is what our recommendations are based upon)**

In [28]:
print(svd.user_similarity(0,2)) # UserID 1 and UserID 3 (the userIDs are shifted by one in this dataset)
print(svd.item_similarity(0,3113)) # Toy Story and Toy Story 2
print(svd.user_item_similarity(0,0)) # UserID 1 and Toy Story

0.0226617045752
0.0969848572946
-0.0115120500119
